<a href="https://colab.research.google.com/github/shuvankarroy/17_flower_classification_cnn/blob/master/Flower_classification_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import required modules

In [0]:
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np
from google.colab import drive

## Steps needed to be excute for google drive

In [0]:
drive.mount('/content/drive')
train_data_dir = '/content/drive/My Drive/flowers_classified_data/train_data'
validation_data_dir = '/content/drive/My Drive/flowers_classified_data/validation_data'

## Steps needed to be executed for local computer execution

In [0]:
train_data_dir = 'flowers_classified_data\\train_data'
validation_data_dir = 'flowers_classified_data\\validation_data'

## Set up required variables

In [0]:
# initialize classes with 17 different classes
classes = ['Tulip', 'Snowdrop', 'LilyValley', 'Bluebell', 'Crocus',
           'Iris', 'Tigerlily', 'Daffodil', 'Fritillary', 'Sunflower',
           'Daisy', 'ColtsFoot', 'Dandelion', 'Cowslip', 'Buttercup',
           'Windflower', 'Pansy']

batch_size = 32
nb_classes = len(classes)

img_rows, img_cols = 150, 150
channels = 3
nb_valid = 10


nb_train_samples = 1190
nb_val_samples = 170
nb_epoch = 50

result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)


## Create validation data (assumption : all data is initially stored in "/flowers_classified_data/train_data" directory)

In [0]:
import os, random
dataSet = os.getcwd()+'\\flowers_classified_data'
selected = []
count = 0

for class_name in classes:
    print(class_name)
    while(True):
        srcpath = dataSet+'\\train_data\\'+class_name
        destpath = dataSet+'\\validation_data\\'+class_name
        fileName = random.choice([x for x in os.listdir(srcpath+'\\') if os.path.isfile(os.path.join(srcpath+'\\', x))]) 
        #print(fileName)
        if(count < nb_valid and fileName not in selected):
            selected.append(fileName)
            print(srcpath+'\\'+fileName)
            #os.rename(srcpath+'\\'+fileName, destpath+'\\'+fileName)
            count += 1
            #print(selected)
        elif(count >= nb_valid):
            count = 0
            print(selected)
            selected = []
            break

In [0]:
if __name__ == '__main__':

    input_tensor = Input(shape=(img_rows, img_cols, channels))
    vgg16 = VGG16(include_top=False, weights=None, input_tensor=input_tensor)


    top_model = Sequential()
    top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(nb_classes, activation='softmax'))

    # VGG16
    model = Model(input=vgg16.input, output=top_model(vgg16.output))

    # VGG16 model compilation
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)
    
    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)
    
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples//batch_size,
        nb_epoch=nb_epoch, 
        validation_data=validation_generator,
        validation_steps=nb_val_samples//batch_size,
        use_multiprocessing=True)

    #model.save_weights(os.path.join(result_dir, 'vgg_scratch.h5'))
    #save_history(history, os.path.join(result_dir, 'history_vgg_scratch.txt'))